# Méthodologie de travail

Étape 1 — Créer le dépôt GitHub

In [ ]:
# Lien du dépôt : https://github.com/inza-bamba/cnn-catsdogs-InzaBamba.git

Étape 2 — Connecter Colab à GitHub

In [ ]:
# dépôt cnn-catsdogs-InzaBamba

Étape 3 — Préparer l’environnement dans Colab

In [1]:
# Installer les dépendances
!pip install torch torchvision tqdm scikit-learn